># Day 4: Scratchcards
>
>The gondola takes you up. Strangely, though, the ground doesn't seem to be coming with you; you're not climbing a mountain. As the circle of Snow Island recedes below you, an entire new landmass suddenly appears above you! The gondola carries you to the surface of the new island and lurches into the station.
>
>As you exit the gondola, the first thing you notice is that the air here is much warmer than it was on Snow Island. It's also quite humid. Is this where the water source is?
>
>The next thing you notice is an Elf sitting on the floor across the station in what seems to be a pile of colorful square cards.
>
>"Oh! Hello!" The Elf excitedly runs over to you. "How may I be of service?" You ask about water sources.
>
>"I'm not sure; I just operate the gondola lift. That does sound like something we'd have, though - this is Island Island, after all! I bet the gardener would know. He's on a different island, though - er, the small kind surrounded by water, not the floating kind. We really need to come up with a better naming scheme. Tell you what: if you can help me with something quick, I'll let you borrow my boat and you can go visit the gardener. I got all these scratchcards as a gift, but I can't figure out what I've won."
>
>The Elf leads you over to the pile of colorful cards. There, you discover dozens of scratchcards, all with their opaque covering already scratched off. Picking one up, it looks like each card has two lists of numbers separated by a vertical bar (|): a list of winning numbers and then a list of numbers you have. You organize the information into a table (your puzzle input).
>
>As far as the Elf has been able to figure out, you have to figure out which of the numbers you have appear in the list of winning numbers. The first match makes the card worth one point and each match after the first doubles the point value of that card.
>
>For example:
>```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
>```

## Read input

When I look at a line from today's input:

```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
```

I immediately see that there's three parts: the card's metadata (or its number), the winning numbers and our numbers. Metadata is separated from the rest with a colon `:` and the sets of numbers are separated by pipe `|`.

Even before I look at what I'm actually supposed to calculate today, I know what information I want from this.

I split the line from these separators and with regex, find all the numbers in individual parts. I store the scratch numbers in sets (more of why below) and return a tuple with card's number and inside a tuple of those number sets.

In [66]:
from utils import read_input
import re


def transformer(line):
    card_info, numbers = line.split(': ')
    card_number = int(re.findall('(\d+)', card_info)[0])
    winning, ours = numbers.split(' | ')
    winning_numbers = set([int(n) for n in re.findall(r'(\d+)', winning)])
    our_numbers = set([int(n) for n in re.findall(r'(\d+)', ours)])
    return (card_number, (winning_numbers, our_numbers))

scratch_cards = read_input(4, transformer)

## Part 1

>In the above example, card 1 has five winning numbers (41, 48, 83, 86, and 17) and eight numbers you have (83, 86, 6, 31, 17, 9, 48, and 53). Of the numbers you have, four of them (48, 83, 17, and 86) are winning numbers! That means card 1 is worth 8 points (1 for the first match, then doubled three times for each of the three matches after the first).
>
>```
    Card 2 has two winning numbers (32 and 61), so it is worth 2 points.
    Card 3 has two winning numbers (1 and 21), so it is worth 2 points.
    Card 4 has one winning number (84), so it is worth 1 point.
    Card 5 has no winning numbers, so it is worth no points.
    Card 6 has no winning numbers, so it is worth no points.
>```
>
>So, in this example, the Elf's pile of scratchcards is worth 13 points.
>
>Take a seat in the large pile of colorful cards. How many points are they worth in total?

### Detour to the world of sets

I mentioned in the read input phase that I stored those sequences of numbers in sets. The reason for that is that [sets](https://docs.python.org/3/tutorial/datastructures.html#sets) have wonderful properties.

From the docs: _"A set is an unordered collection with no duplicate elements."_ and it's perfect for our puzzle today. We don't care about the order of numbers in either the winning numbers nor our own and none of the numbers can appear twice.

Today, we're using [intersection](https://docs.python.org/3/library/stdtypes.html#frozenset.intersection) which takes any number of sets combined with ambersand (`&`) and returns only those that are present in every set.

Another benefit of sets that we don't need today is that checking if something is in a set is efficient. In data structure lingo, we'd say it's O(1), in other words the time it takes to check if an item is in the set is constant and does not depend on the size of the set. Where-as checking the same in a list is O(n) which means the time grows linearly in regard to the size of the list. I'm pretty sure we'll get to use that in a later puzzle this year.

### Back to the puzzle

We need to find out which of our numbers were included in winning numbers: intersect the two. And take the length of that resulting set.

To calculate points in a scheme that looks like this:

|How many right?|Points|
|--|--|
|0|0|
|1|1|
|2|2|
|3|4|
|4|8|
|5|16|

We can identify a pattern: it's the 2's exponents, offset by one. 2^0 = 1, 2^1 = 2, 2^2 = 4 and so on.

We do need to be a bit careful though since 2^-1 = 0.5 so I'm filtering out the point scoring to happen only when there are any winning numbers.

In Python, we can raise a number to a power with `**` as in `2**exp` which is same as 2^exp in written math.

In [55]:
def calculate_points(scratch_cards):
    points = 0
    for _id, (winning, our) in scratch_cards:
        numbers_that_won = winning & our
        if numbers_that_won:
            points += 2 ** (len(numbers_that_won) - 1)
    return points

In [56]:
part_1 = calculate_winnings(scratch_cards)
print(f'Solution: {part_1}')
assert part_1 == 26218

Solution: 26218


## Part 2

>Just as you're about to report your findings to the Elf, one of you realizes that the rules have actually been printed on the back of every card this whole time.
>
>There's no such thing as "points". Instead, scratchcards only cause you to win more scratchcards equal to the number of winning numbers you have.
>
>Specifically, you win copies of the scratchcards below the winning card equal to the number of matches. So, if card 10 were to have 5 matching numbers, you would win one copy each of cards 11, 12, 13, 14, and 15.
>
>Copies of scratchcards are scored like normal scratchcards and have the same card number as the card they copied. So, if you win a copy of card 10 and it has 5 matching numbers, it would then win a copy of the same cards that the original card 10 won: cards 11, 12, 13, 14, and 15. This process repeats until none of the copies cause you to win any more cards. (Cards will never make you copy a card past the end of the table.)
>
>This time, the above example goes differently:
>```
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
>```
>- Card 1 has four matching numbers, so you win one copy each of the next four cards: cards 2, 3, 4, and 5.
>- Your original card 2 has two matching numbers, so you win one copy each of cards 3 and 4.
>- Your copy of card 2 also wins one copy each of cards 3 and 4.
>- Your four instances of card 3 (one original and three copies) have two matching numbers, so you win four copies each of cards 4 and 5.
>- Your eight instances of card 4 (one original and seven copies) have one matching number, so you win eight copies of card 5.
>- Your fourteen instances of card 5 (one original and thirteen copies) have no matching numbers and win no more cards.
>- Your one instance of card 6 (one original) has no matching numbers and wins no more cards.
>
>Once all of the originals and copies have been processed, you end up with 1 instance of card 1, 2 instances of card 2, 4 instances of card 3, 8 instances of card 4, 14 instances of card 5, and 1 instance of card 6. In total, this example pile of scra
tchcards causes you to ultimately have 30 scratchcards!
>
>Process all of the original and copied scratchcards until no more scratchcards are won. Including the original set of scratchcards, how many total scratchcards do you end up with?

In the second part, most of the code is the same. Here, I initialize a dictionary that keeps track of how many of each scratch card there are. I use a [dict comprehension](https://peps.python.org/pep-0274/) to initialize each value to 1.

Then, we calculate the numbers that won as we did in part 1 and for each of those wins, we increment the following card counts by one.

This is the type of puzzle that in the back of my head, I'm thinking: "There is probably a mathematical equation for this" but since I'm no mathematician, I just keep looping.

As none of the winnings affect cards that came before them, we never have to worry about re-running any previous cards.

To calculate the final answer, we sum up all the values in the counter.

In [71]:
def process_cards(scratch_cards):
    counts = { card_id: 1 for card_id, _ in scratch_cards }
    for card_id, (winning, our) in scratch_cards:
        numbers_that_won = winning & our
        winning_count = len(numbers_that_won)

        # I sometimes wish Python had a `times(n)` function 
        # to do something n times. `range(n)` doesn't have
        # the same semantic feel
        
        for _ in range(counts[card_id]):
            for i in range(card_id + 1, card_id + winning_count + 1):
                counts[i] += 1
    return counts

In [72]:
card_counts = process_cards(scratch_cards)
part_2 = sum(c for c in card_counts.values())
print(f'Solution: {part_2}')
assert part_2 == 9997537

Solution: 9997537


## Two stars

8 out of 50. Today was a lot of fun. Working with sets and dictionaries is always a joy in Python.

## Additional Markdown styling

This is not part of the puzzle, just here to keep the table styling better

In [63]:
%%html
<style>
table {margin-left: 0 !important;}
</style>